## Solar Calculations

In [ ]:
%cd /app

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import datetime
from typing import Final
import pvlib as pv
import folium

from src.solar import get_time_series

In [ ]:
## terrace and building positions
LATITUDE: Final[float] = 46.202836
LONGITUDE: Final[float] = 6.151757
ALTITUDE: Final[float] = 375.06  # Altitude in meters
TIMEZONE: Final[str] = "Europe/Zurich"
DATE_FORMAT: Final[str] = "%Y-%m-%d"
TERRACE_CORNERS: Final[dict[str, tuple[float, float]]] = {
    "NW": (46.202887, 6.151729),
    "NE": (46.202878, 6.151759),
    "SW": (46.202806, 6.151667),
    "SE": (46.202800, 6.151700),
}

BUILDING_CORNERS: Final[dict[str, tuple[float, float]]] = {
    "NW": (46.202100, 6.151110),
    "NE": (46.202887, 6.151729),
    "SW": (46.202778, 6.151389),
    "SE": (46.202806, 6.151667),
}

BUILDING_HEIGHT = 399.79 - 374.57


In [ ]:
def translate_point(lat, long, length, bearing):

    # convert to radians
    lat = np.radians(lat)
    long = np.radians(long)
    bearing = np.radians(bearing)

    # Earth radius in meters
    R = 6371000

    frac_distance = length / R

    lat2 = np.arcsin(np.sin(lat) * np.cos(frac_distance) + np.cos(lat) * np.sin(frac_distance) * np.cos(bearing))
    long2 = long + np.arctan2(np.sin(bearing) * np.sin(frac_distance) * np.cos(lat),
                              np.cos(frac_distance) - np.sin(lat) * np.sin(lat2))
    
    return np.degrees(lat2), np.degrees(long2)

In [ ]:
def shadow_bearing(azimuth):
    return (azimuth + 180) % 360

In [ ]:
shadow_bearing(np.array([180, 270, 0, 90]))

In [ ]:
BUILDING_CORNERS['NW'] = translate_point(BUILDING_CORNERS['NE'][0], BUILDING_CORNERS['NE'][1], 20, 293)
BUILDING_CORNERS['SW'] = translate_point(BUILDING_CORNERS['SE'][0], BUILDING_CORNERS['SE'][1], 20, 293)

In [ ]:
# m = folium.Map(location=[LATITUDE, LONGITUDE], zoom_start=18)

# coords_terrace = [
#     TERRACE_CORNERS["NW"],
#     TERRACE_CORNERS["NE"],
#     TERRACE_CORNERS["SE"],
#     TERRACE_CORNERS["SW"],
#     TERRACE_CORNERS["NW"],  # Close the loop
# ]

# folium.Polygon(
#     locations=coords_terrace,
#     color="blue",
#     fill=True,
#     fill_color="blue",
#     fill_opacity=0.3
# ).add_to(m)


# coords_building = [
#     BUILDING_CORNERS["NW"],
#     BUILDING_CORNERS["NE"],
#     BUILDING_CORNERS["SE"],
#     BUILDING_CORNERS["SW"],
#     BUILDING_CORNERS["NW"],  # Close the loop
# ]

# folium.Polygon(
#     locations=coords_building,
#     color="red",
#     fill=True,
#     fill_color="red",
#     fill_opacity=0.3
# ).add_to(m)

# mcoords_terrace = [
#     TERRACE_CORNERS["NW"],
#     TERRACE_CORNERS["NE"],
#     TERRACE_CORNERS["SE"],
#     TERRACE_CORNERS["SW"],
#     TERRACE_CORNERS["NW"],  # Close the loop
# ]

# folium.Polygon(
#     locations=coords_terrace,
#     color="blue",
#     fill=True,
#     fill_color="blue",
#     fill_opacity=0.3
# ).add_to(m)


# coords_building = [
#     BUILDING_CORNERS["NW"],
#     BUILDING_CORNERS["NE"],
#     BUILDING_CORNERS["SE"],
#     BUILDING_CORNERS["SW"],
#     BUILDING_CORNERS["NW"],  # Close the loop
# ]

# folium.Polygon(
#     locations=coords_building,
#     color="red",
#     fill=True,
#     fill_color="red",
#     fill_opacity=0.3
# ).add_to(m)

# m

In [ ]:
time_series = get_time_series(datetime.datetime(2025, 3, 17))

In [ ]:
time_series[0]

In [ ]:
solar_position_dfs = []

for corner, (lat, long) in BUILDING_CORNERS.items():
    solar_position_df = pv.solarposition.get_solarposition(
        time = time_series,
        latitude = lat, 
        longitude= long,
        altitude=ALTITUDE,
        pressure=None,
        temperature=12,
        method="nrel_numpy"
    )
    solar_position_df['corner'] = corner
    solar_position_dfs.append(solar_position_df)

solar_position_df = pd.concat(solar_position_dfs)

In [ ]:
solar_position_df.columns

In [ ]:
def shadow_length(solar_elevation, height):
    return height / np.tan(np.radians(solar_elevation))



solar_position_df['shadow_length'] = shadow_length(solar_position_df['elevation'], BUILDING_HEIGHT)
solar_position_df['shadow_bearing'] = shadow_bearing(solar_position_df['azimuth'])



In [ ]:
# Extract data for the northwest corner
nw_corner_data = solar_position_df[solar_position_df['corner'] == 'NW'].copy()
nw_corner_data.index = pd.to_datetime(nw_corner_data.index)

# Filter for positive elevation (sun above horizon) and positive shadow lengths
daylight_data = nw_corner_data[nw_corner_data['elevation'] > 0]
valid_shadows = daylight_data[daylight_data['shadow_length'] > 0]

# Create figure and axis
fig, ax = plt.subplots(figsize=(14, 8))

# Plot shadow length vs time
ax.plot(valid_shadows.index, valid_shadows['shadow_length'], 'b-', linewidth=2, label='Shadow Length')

# Add shaded area for daylight hours
if not daylight_data.empty:
    ax.axvspan(daylight_data.index.min(), daylight_data.index.max(), 
               color='yellow', alpha=0.2, label='Daylight')

# Add min shadow length line with annotation
if not valid_shadows.empty:
    min_shadow = valid_shadows['shadow_length'].min()
    min_time = valid_shadows[valid_shadows['shadow_length'] == min_shadow].index[0]
    
    ax.axhline(y=min_shadow, color='r', linestyle='--', 
               label=f'Min Shadow Length: {min_shadow:.2f}m')
    
    # Add annotation for minimum shadow
    ax.annotate(f'Min: {min_shadow:.2f}m at {min_time.strftime("%H:%M")}',
                xy=(min_time, min_shadow),
                xytext=(min_time, min_shadow + 50),  # Offset text 50m above the min point
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5),
                bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.8))

# Format the plot
ax.set_title('Shadow Length from Northwest Corner of Building Throughout the Day', fontsize=14)
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Shadow Length (meters)', fontsize=12)
ax.set_ylim(bottom=0, top=100)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right')

# set x limit between 6am and 6pm

# Format x-axis to show hours
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:

solar_position_dfs = []

for corner, (lat, long) in BUILDING_CORNERS.items():

    subset_df = solar_position_df[solar_position_df['corner'] == corner].copy()
    lat2, long2 = translate_point(lat, long, subset_df['shadow_length'], subset_df['shadow_bearing'])

    subset_df['shadow_lat'] = lat2
    subset_df['shadow_lon'] = long2

    solar_position_dfs.append(subset_df)

shadow_df = pd.concat(solar_position_dfs)

In [ ]:
shadow_df

In [ ]:
# Create a new map centered at the same location
shadow_map = folium.Map(location=[LATITUDE, LONGITUDE], zoom_start=18)

# Add terrace polygon
terrace_coords = [
    TERRACE_CORNERS["NW"],
    TERRACE_CORNERS["NE"],
    TERRACE_CORNERS["SE"],
    TERRACE_CORNERS["SW"],
    TERRACE_CORNERS["NW"],  # Close the loop
]

folium.Polygon(
    locations=terrace_coords,
    color="blue",
    fill=True,
    fill_color="blue",
    fill_opacity=0.3,
    tooltip="Terrace"
).add_to(shadow_map)

# Add building polygon
building_coords = [
    BUILDING_CORNERS["NW"],
    BUILDING_CORNERS["NE"],
    BUILDING_CORNERS["SE"],
    BUILDING_CORNERS["SW"],
    BUILDING_CORNERS["NW"],  # Close the loop
]

folium.Polygon(
    locations=building_coords,
    color="red",
    fill=True,
    fill_color="red",
    fill_opacity=0.3,
    tooltip="Building"
).add_to(shadow_map)

# Select a specific time (noon for example)
selected_time = pd.Timestamp('2025-03-17 16:45:00+0100', tz='Europe/Zurich')
shadow_at_time = shadow_df[shadow_df.index == selected_time]

# Create shadow polygon for the selected time
if not shadow_at_time.empty:
    # Extract shadow points for each building corner at the selected time
    shadow_points = []
    
    for corner in ['NW', 'NE', 'SE', 'SW']:
        corner_shadow = shadow_at_time[shadow_at_time['corner'] == corner]
        if not corner_shadow.empty:
            shadow_points.append((corner_shadow['shadow_lat'].values[0], corner_shadow['shadow_lon'].values[0]))
    
    # Add building corners to create a complete shadow polygon if we have all 4 points
    if len(shadow_points) == 4:
        # Add the first point again to close the polygon
        shadow_points.append(shadow_points[0])
        
        folium.Polygon(
            locations=shadow_points,
            color="black",
            fill=True,
            fill_color="black",
            fill_opacity=0.5,
            tooltip=f"Shadow at {selected_time.strftime('%H:%M')}"
        ).add_to(shadow_map)
        
        # Add markers for each shadow point
        # for i, point in enumerate(shadow_points[:-1]):
        #     folium.Marker(
        #         location=point,
        #         tooltip=f"Shadow of {['NW', 'NE', 'SE', 'SW'][i]} corner at {selected_time.strftime('%H:%M')}",
        #         icon=folium.Icon(color="black", icon="map-pin")
        #     ).add_to(shadow_map)


# add code that connects the corners of the building to the shadow points
for corner in ['NW', 'NE', 'SE', 'SW']:
    corner_lat, corner_long = BUILDING_CORNERS[corner]
    shadow_lat, shadow_long = shadow_df[(shadow_df.index == selected_time) & (shadow_df['corner'] == corner)][['shadow_lat', 'shadow_lon']].values[0]
    
    folium.PolyLine(
        locations=[(corner_lat, corner_long), (shadow_lat, shadow_long)],
        color="black",
        tooltip=f"Shadow of {corner} corner at {selected_time.strftime('%H:%M')}"
    ).add_to(shadow_map)

shadow_map

In [ ]:
from src.geometry import _get_intersection, _get_convex_hull

terrace_df = pd.DataFrame(terrace_coords, columns=['lat', 'lon'])
shadow_df = shadow_df.loc[shadow_df.index == selected_time, ['shadow_lat', 'shadow_lon']].copy()
building_df = pd.DataFrame(BUILDING_CORNERS.values(), columns=['shadow_lat', 'shadow_lon'])
shadow_df = pd.concat([building_df, shadow_df])
shadow_df['time'] = selected_time
shadow_df['egid'] = 'a'




In [ ]:
shadow_hull_df = pd.DataFrame(_get_convex_hull(shadow_df[['shadow_lat', 'shadow_lon']].values), columns=['lat', 'lon'])

In [ ]:
intersection_df, _ = _get_intersection(shadow_hull_df.values, terrace_df.values)
intersection_df = pd.DataFrame(tuple(intersection_df.exterior.coords), columns=['lat', 'lon'])

In [ ]:
# Create a map centered at the same location
intersection_map = folium.Map(location=[LATITUDE, LONGITUDE], zoom_start=19)

# Add terrace polygon
folium.Polygon(
    locations=terrace_df.values,
    color="blue",
    fill=True,
    fill_color="blue",
    fill_opacity=0.3,
    tooltip="Terrace"
).add_to(intersection_map)

# Add building polygon
building_coords = [
    BUILDING_CORNERS["NW"],
    BUILDING_CORNERS["NE"],
    BUILDING_CORNERS["SE"],
    BUILDING_CORNERS["SW"],
    BUILDING_CORNERS["NW"],  # Close the loop
]
folium.Polygon(
    locations=building_coords,
    color="red",
    fill=True,
    fill_color="red",
    fill_opacity=0.3,
    tooltip="Building"
).add_to(intersection_map)

# Add shadow hull polygon
shadow_hull_coords = shadow_hull_df.values.tolist()
shadow_hull_coords.append(shadow_hull_coords[0])  # Close the loop
folium.Polygon(
    locations=shadow_hull_coords,
    color="black",
    fill=True,
    fill_color="black",
    fill_opacity=0.15,
    tooltip=f"Shadow Hull at {selected_time.strftime('%H:%M')}"
).add_to(intersection_map)

# Add intersection polygon
folium.Polygon(
    locations=intersection_df.values,
    color="purple",
    fill=True,
    fill_color="purple",
    fill_opacity=0.7,
    tooltip=f"Intersection at {selected_time.strftime('%H:%M')}"
).add_to(intersection_map)

# Add a time marker
folium.Marker(
    location=[LATITUDE + 0.0005, LONGITUDE],
    icon=folium.DivIcon(
        icon_size=(150, 36),
        icon_anchor=(75, 18),
        html=f'<div style="font-size: 14pt; background-color: white; padding: 5px; border-radius: 5px;">{selected_time.strftime("%H:%M")}</div>'
    )
).add_to(intersection_map)

intersection_map